In [1]:
from pocketknife.database import (connect_database, read_from_database)
import pandas as pd

In [2]:
with connect_database() as con:
    query = """
        select * from zefix.base limit 100
    """
    df = read_from_database(connection=con, query=query, )

ValueError: No value set for configuration key DATABASE -> LINKMED -> NAME.